# 🚀 arXiv Scraper - Lab 1 Data Science
## Student ID: 23127240

**Yêu cầu Lab 1:**
- Testbed: Google Colab CPU-only mode
- Đo wall time (end-to-end)
- Đo memory footprint (max RAM, disk usage)
- Thu thập: TeX sources, metadata, references
- Remove figures để giảm kích thước

---

## ⚙️ BƯỚC 1: Kiểm tra Runtime (CPU-only)

In [ ]:
# Kiểm tra runtime type (phải là CPU theo yêu cầu Lab 1)
import psutil
import platform

print("=" * 60)
print("THÔNG TIN RUNTIME")
print("=" * 60)
print(f"OS: {platform.system()} {platform.release()}")
print(f"CPU cores: {psutil.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
print(f"Disk: {psutil.disk_usage('/').total / (1024**3):.2f} GB")
print("=" * 60)

# Đảm bảo không có GPU (theo yêu cầu CPU-only)
try:
    import torch
    if torch.cuda.is_available():
        print("⚠️ WARNING: GPU detected! Lab yêu cầu CPU-only mode")
        print("Chuyển sang Runtime > Change runtime type > Hardware accelerator > None")
    else:
        print("✅ CPU-only mode - Đúng yêu cầu Lab 1")
except:
    print("✅ CPU-only mode - Đúng yêu cầu Lab 1")

## 📥 BƯỚC 2: Clone Source Code từ GitHub

In [ ]:
# Clone repository
!git clone https://github.com/nhutphansayhi/ScrapingDataNew.git
%cd ScrapingDataNew/23127240
!ls -la

## 📦 BƯỚC 3: Cài đặt Dependencies

In [ ]:
# Cài đặt thư viện cần thiết
!pip install -q arxiv requests beautifulsoup4 bibtexparser psutil

# Verify installation
import arxiv
import requests
from bs4 import BeautifulSoup
import bibtexparser
import psutil
import json
import time

print("✅ Tất cả thư viện đã được cài đặt!")

## 📊 BƯỚC 4: Setup Performance Monitor

In [ ]:
import psutil
import time
import os
import json
from datetime import datetime

class PerformanceMonitor:
    """Monitor để đo wall time và memory footprint theo yêu cầu Lab 1"""
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.initial_disk_mb = 0
        self.max_ram_mb = 0
        self.max_disk_mb = 0
        self.paper_times = []
        
    def start(self):
        """Bắt đầu đo wall time (end-to-end)"""
        self.start_time = time.time()
        self.initial_disk_mb = psutil.disk_usage('/').used / (1024**2)
        initial_ram = psutil.virtual_memory().used / (1024**2)
        
        print("\n" + "=" * 70)
        print("🚀 Bắt đầu scraping: {}".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
        print("=" * 70)
        print("📊 Initial disk: {:.2f} MB".format(self.initial_disk_mb))
        print("💾 Initial RAM: {:.2f} MB".format(initial_ram))
        print("=" * 70)
        
    def update_metrics(self, paper_id=None, paper_time=None):
        """Cập nhật metrics trong quá trình scraping"""
        ram_mb = psutil.virtual_memory().used / (1024**2)
        self.max_ram_mb = max(self.max_ram_mb, ram_mb)
        
        disk_mb = psutil.disk_usage('/').used / (1024**2)
        self.max_disk_mb = max(self.max_disk_mb, disk_mb)
        
        if paper_id and paper_time is not None:
            self.paper_times.append({
                'paper_id': paper_id,
                'time_seconds': paper_time
            })
        
    def finish(self, output_dir=None):
        """Kết thúc và tính toán metrics"""
        self.end_time = time.time()
        total_time = self.end_time - self.start_time
        disk_increase = self.max_disk_mb - self.initial_disk_mb
        
        print("\n" + "=" * 70)
        print("📊 PERFORMANCE METRICS (theo yêu cầu Lab 1)")
        print("=" * 70)
        
        # Wall time
        print("\n⏱️  WALL TIME (End-to-End):")
        print("   Total: {:.2f} seconds ({:.2f} minutes)".format(total_time, total_time/60))
        
        if self.paper_times:
            avg_time = sum(p['time_seconds'] for p in self.paper_times) / len(self.paper_times)
            print("   Average per paper: {:.2f} seconds".format(avg_time))
            print("   Papers processed: {}".format(len(self.paper_times)))
        
        # Memory footprint
        print("\n💾 MEMORY FOOTPRINT:")
        print("   Maximum RAM used: {:.2f} MB ({:.2f} GB)".format(self.max_ram_mb, self.max_ram_mb/1024))
        current_ram = psutil.virtual_memory().used / (1024**2)
        print("   Current RAM: {:.2f} MB".format(current_ram))
        
        # Disk usage
        print("\n💿 DISK USAGE:")
        print("   Maximum disk used: {:.2f} MB ({:.2f} GB)".format(self.max_disk_mb, self.max_disk_mb/1024))
        print("   Disk increase: {:.2f} MB ({:.2f} GB)".format(disk_increase, disk_increase/1024))
        
        # Calculate output size
        output_size_mb = 0
        if output_dir and os.path.exists(output_dir):
            total_size = sum(
                os.path.getsize(os.path.join(dp, f))
                for dp, dn, filenames in os.walk(output_dir)
                for f in filenames
            )
            output_size_mb = total_size / (1024**2)
            print("   Output data size: {:.2f} MB ({:.2f} GB)".format(output_size_mb, output_size_mb/1024))
        
        print("=" * 70)
        
        return {
            'testbed': 'Google Colab CPU-only',
            'total_wall_time_seconds': total_time,
            'total_wall_time_minutes': total_time / 60,
            'total_wall_time_hours': total_time / 3600,
            'max_ram_mb': self.max_ram_mb,
            'max_ram_gb': self.max_ram_mb / 1024,
            'disk_increase_mb': disk_increase,
            'disk_increase_gb': disk_increase / 1024,
            'output_size_mb': output_size_mb,
            'output_size_gb': output_size_mb / 1024,
            'papers_processed': len(self.paper_times),
            'avg_time_per_paper': sum(p['time_seconds'] for p in self.paper_times) / len(self.paper_times) if self.paper_times else 0,
            'timestamp': datetime.now().isoformat()
        }

# Khởi tạo monitor
monitor = PerformanceMonitor()
print("✅ Performance Monitor đã sẵn sàng!")

## 🔧 BƯỚC 5: Chạy Scraper với Performance Monitoring

**Lưu ý:** Script sẽ tự động:
- Scrape metadata từ arXiv API
- Download TeX sources (.tar.gz)
- Remove figures (png, jpg, pdf, eps)
- Crawl references từ Semantic Scholar
- Lưu theo cấu trúc yêu cầu Lab 1

In [ ]:
import subprocess
import sys
import os

# BẮT ĐẦU ĐO WALL TIME
monitor.start()

try:
    print("🔄 Đang chạy scraper...")
    print("\nQuy trình (theo đề bài Lab 1):")
    print("  1️⃣  Entry Discovery: arXiv API")
    print("  2️⃣  Source Download: .tar.gz extraction")
    print("  3️⃣  Figure Removal: Xóa png, jpg, pdf, eps")
    print("  4️⃣  Reference Crawling: Semantic Scholar API")
    print("  5️⃣  Data Organization: tex/, metadata.json, references.json")
    print("\n" + "-" * 70)
    
    # Di chuyển vào thư mục src
    os.chdir('/content/ScrapingDataNew/23127240/src')
    
    # Chạy scraper bằng subprocess (chạy như terminal command)
    result = subprocess.run(
        ['python3', 'main.py'],
        capture_output=True,
        text=True,
        timeout=7200  # 2 hours timeout
    )
    
    # Hiển thị output
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)
    
    # Check exit code
    if result.returncode != 0:
        print(f"\n⚠️  Scraper thoát với code: {result.returncode}")
    else:
        print("\n✅ Scraper hoàn tất thành công!")
    
    # Update metrics
    monitor.update_metrics()
    
    # Về thư mục gốc
    os.chdir('/content/ScrapingDataNew/23127240')
    
except subprocess.TimeoutExpired:
    print("\n⚠️  Timeout: Scraping vượt quá 2 giờ")
    os.chdir('/content/ScrapingDataNew/23127240')
except KeyboardInterrupt:
    print("\n⚠️  Scraping bị ngắt bởi user")
    os.chdir('/content/ScrapingDataNew/23127240')
except Exception as e:
    print(f"\n❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()
    os.chdir('/content/ScrapingDataNew/23127240')
finally:
    # KẾT THÚC ĐO WALL TIME
    metrics = monitor.finish(output_dir="23127240_data")
    
    # Lưu metrics
    with open('performance_metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    print("\n💾 Metrics đã lưu vào: performance_metrics.json")

### 🔄 Cách 2: Chạy trực tiếp bằng terminal (ĐƠN GIẢN NHẤT)

Nếu cách trên lỗi, dùng cách này:

In [ ]:
# BẮT ĐẦU ĐO WALL TIME
monitor.start()

# Di chuyển vào thư mục src
%cd /content/ScrapingDataNew/23127240/src

# Chạy scraper
!python3 main.py

# Về thư mục gốc
%cd /content/ScrapingDataNew/23127240

# KẾT THÚC ĐO WALL TIME
metrics = monitor.finish(output_dir="23127240_data")

# Lưu metrics
import json
with open('performance_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n💾 Metrics đã lưu vào: performance_metrics.json")

## 📁 BƯỚC 6: Kiểm tra Cấu trúc Dữ liệu

In [ ]:
import os
import json

def verify_data_structure(data_dir="23127240_data"):
    """Kiểm tra cấu trúc dữ liệu theo yêu cầu Lab 1"""
    
    print("=" * 70)
    print("📁 KIỂM TRA CẤU TRÚC DỮ LIỆU")
    print("=" * 70)
    
    if not os.path.exists(data_dir):
        print(f"❌ Thư mục {data_dir} không tồn tại!")
        return
    
    papers = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    print(f"\n📊 Tổng số papers: {len(papers)}")
    
    stats = {
        'total_papers': len(papers),
        'papers_with_tex': 0,
        'papers_with_metadata': 0,
        'papers_with_references': 0,
        'total_versions': 0,
        'total_tex_files': 0,
        'total_bib_files': 0,
        'total_references': 0
    }
    
    # Check first 10 papers in detail
    for paper_id in sorted(papers)[:10]:
        paper_path = os.path.join(data_dir, paper_id)
        print(f"\n📄 {paper_id}:")
        
        # Check tex folder
        tex_path = os.path.join(paper_path, "tex")
        if os.path.exists(tex_path):
            versions = [d for d in os.listdir(tex_path) if os.path.isdir(os.path.join(tex_path, d))]
            stats['papers_with_tex'] += 1
            stats['total_versions'] += len(versions)
            print(f"   ✅ tex/ - {len(versions)} version(s)")
            
            # Count .tex and .bib files
            for version in versions:
                version_path = os.path.join(tex_path, version)
                for root, dirs, files in os.walk(version_path):
                    stats['total_tex_files'] += len([f for f in files if f.endswith('.tex')])
                    stats['total_bib_files'] += len([f for f in files if f.endswith('.bib')])
        else:
            print(f"   ❌ tex/ missing")
        
        # Check metadata.json
        metadata_path = os.path.join(paper_path, "metadata.json")
        if os.path.exists(metadata_path):
            stats['papers_with_metadata'] += 1
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
                title = metadata.get('title', 'N/A')
                print(f"   ✅ metadata.json - {title[:60]}...")
        else:
            print(f"   ❌ metadata.json missing")
        
        # Check references.json
        ref_path = os.path.join(paper_path, "references.json")
        if os.path.exists(ref_path):
            stats['papers_with_references'] += 1
            with open(ref_path, 'r') as f:
                refs = json.load(f)
                stats['total_references'] += len(refs)
                print(f"   ✅ references.json - {len(refs)} reference(s)")
        else:
            print(f"   ❌ references.json missing")
    
    # Count all papers
    for paper_id in papers:
        paper_path = os.path.join(data_dir, paper_id)
        if os.path.exists(os.path.join(paper_path, "tex")):
            stats['papers_with_tex'] += 1
        if os.path.exists(os.path.join(paper_path, "metadata.json")):
            stats['papers_with_metadata'] += 1
        if os.path.exists(os.path.join(paper_path, "references.json")):
            stats['papers_with_references'] += 1
    
    print("\n" + "=" * 70)
    print("📊 STATISTICS SUMMARY")
    print("=" * 70)
    for key, value in stats.items():
        print(f"   {key}: {value}")
    
    # Calculate success rates
    if stats['total_papers'] > 0:
        print("\n📈 SUCCESS RATES:")
        print(f"   TeX success: {stats['papers_with_tex']/stats['total_papers']*100:.1f}%")
        print(f"   Metadata success: {stats['papers_with_metadata']/stats['total_papers']*100:.1f}%")
        print(f"   References success: {stats['papers_with_references']/stats['total_papers']*100:.1f}%")
        if stats['papers_with_references'] > 0:
            avg_refs = stats['total_references'] / stats['papers_with_references']
            print(f"   Avg references per paper: {avg_refs:.1f}")
    
    print("=" * 70)
    
    return stats

# Chạy verification
stats = verify_data_structure()

## 📊 BƯỚC 7: Performance Report cho Report.docx

In [ ]:
# Load performance metrics
with open('performance_metrics.json', 'r') as f:
    metrics = json.load(f)

print("=" * 70)
print("📈 FINAL PERFORMANCE REPORT (copy vào Report.docx)")
print("=" * 70)

print("\n🎯 TESTBED: Google Colab CPU-only mode")
print("-" * 70)

print("\n⏱️  RUNNING TIME:")
print(f"   • Total wall time: {metrics['total_wall_time_seconds']:.2f}s ({metrics['total_wall_time_seconds']/60:.2f} min)")
print(f"   • Average time per paper: {metrics['avg_time_per_paper']:.2f}s")
print(f"   • Papers processed: {metrics['papers_processed']}")

print("\n💾 MEMORY FOOTPRINT:")
print(f"   • Maximum RAM used: {metrics['max_ram_mb']:.2f} MB ({metrics['max_ram_mb']/1024:.2f} GB)")
print(f"   • Maximum disk used: {metrics['max_disk_mb']:.2f} MB ({metrics['max_disk_mb']/1024:.2f} GB)")
if 'disk_increase_mb' in metrics:
    print(f"   • Disk increase: {metrics['disk_increase_mb']:.2f} MB ({metrics['disk_increase_mb']/1024:.2f} GB)")

print("\n📊 DATA STATISTICS:")
if stats:
    print(f"   • Total papers scraped: {stats['total_papers']}")
    print(f"   • Papers with TeX: {stats['papers_with_tex']}")
    print(f"   • Papers with metadata: {stats['papers_with_metadata']}")
    print(f"   • Papers with references: {stats['papers_with_references']}")
    print(f"   • Total versions: {stats['total_versions']}")
    print(f"   • Total .tex files: {stats['total_tex_files']}")
    print(f"   • Total .bib files: {stats['total_bib_files']}")
    print(f"   • Total references: {stats['total_references']}")
    if stats['papers_with_references'] > 0:
        avg_refs = stats['total_references'] / stats['papers_with_references']
        print(f"   • Average references per paper: {avg_refs:.1f}")
    
    # Success rates
    if stats['total_papers'] > 0:
        print(f"\n📈 SUCCESS RATES:")
        print(f"   • Overall success rate: {stats['papers_with_metadata']/stats['total_papers']*100:.1f}%")
        print(f"   • TeX extraction rate: {stats['papers_with_tex']/stats['total_papers']*100:.1f}%")
        print(f"   • Reference crawling rate: {stats['papers_with_references']/stats['total_papers']*100:.1f}%")

print("\n✅ Copy metrics này vào Report.docx!")
print("=" * 70)

## 📥 BƯỚC 8: Download Dữ liệu

In [ ]:
import shutil
from google.colab import files

# Nén dữ liệu
print("📦 Đang nén dữ liệu...")
shutil.make_archive('23127240_data', 'zip', '.', '23127240_data')
print(f"✅ Đã tạo 23127240_data.zip")

# Kiểm tra kích thước
size_mb = os.path.getsize('23127240_data.zip') / (1024**2)
print(f"📊 Kích thước: {size_mb:.2f} MB")

if size_mb > 100:
    print("⚠️ File lớn hơn 100MB, khuyến nghị upload lên Google Drive")
    print("Chạy cell tiếp theo để upload lên Drive")
else:
    print("\n⬇️ Bắt đầu download...")
    files.download('23127240_data.zip')

## ☁️ BƯỚC 9: Upload lên Google Drive (nếu file quá lớn)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy vào Drive
!cp 23127240_data.zip /content/drive/MyDrive/
!cp performance_metrics.json /content/drive/MyDrive/

print("✅ Đã upload vào Google Drive:")
print("   - 23127240_data.zip")
print("   - performance_metrics.json")

---

## 📝 GHI CHÚ

### Yêu cầu Lab 1 đã hoàn thành:
- ✅ Testbed: Google Colab CPU-only mode
- ✅ Wall time measurement (end-to-end)
- ✅ Memory footprint (max RAM, disk usage)
- ✅ Scrape: TeX sources, metadata, references
- ✅ Remove figures để giảm kích thước
- ✅ Cấu trúc theo format yêu cầu

### Rate Limiting:
- Semantic Scholar: 1 req/s, 100 req/5min
- Script có built-in retry mechanism

### Demo Video (≤120s):
1. Setup (15s): Mở Colab, check CPU-only, clone repo
2. Running (45s): Chạy scraper, show logs
3. Results (45s): Performance metrics, verify structure
4. Voice: Giải thích scraper design và reasoning

### Liên hệ:
- Instructor: hlhdang@fit.hcmus.edu.vn

---

**Chúc bạn scraping thành công! 🚀**